In [ ]:
from torch.autograd import Variable
import numpy as np
import tensorflow as tf
import time, math, glob
import scipy.io as sio
import torch
import torch.nn as nn
from math import sqrt
import argparse, os
import torch
import random
import torch.backends.cudnn as cudnn
import torch.optim as optim
from torch.utils.data import DataLoader, TensorDataset
import torch.utils.data as data
import h5py

from __future__ import print_function, division

import torch
import torch.nn as nn
import torch.optim as optim
from torch.optim import lr_scheduler
import numpy as np
import torchvision
from torchvision import datasets, models, transforms
import os
import copy


In [ ]:
import numpy as np
import torch
import torchvision.datasets as dsets
import torchvision.transforms as transforms
from PIL.ImageOps import colorize

In [ ]:
import h5py as h5py
import numpy as np
import tensorflow as tf
from sklearn import metrics
import torchvision.models as models
from PIL import Image
import imageio
import matplotlib.pyplot as plt
import cv2

In [ ]:
from google.colab import drive
drive.mount('/content/drive')
path = "/content/drive/MyDrive"
os.chdir(path)
os.listdir(path)

Mounted at /content/drive


['Colab Notebooks',
 'content',
 'drive',
 'train',
 'validation',
 'Cifar10_Aug',
 '0',
 '5',
 'tensorboard',
 'Cifar10_split',
 'TIP Moire Pattern Removal',
 'data',
 'checkpoint_old',
 'checkpoint']

In [ ]:
data_transforms = {
    'train': transforms.Compose([
        #ransforms.Resize(128),
        transforms.RandomRotation(20),
        transforms.ColorJitter(),
        #transforms.RandomResizedCrop(224),
        transforms.RandomHorizontalFlip(),
        transforms.ToTensor(),
        transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
    ]),
    'val': transforms.Compose([
        #transforms.Resize(128),
        #transforms.CenterCrop(224),
        transforms.ToTensor(),
        transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
    ]),
    'test': transforms.Compose([
        #transforms.Resize(128),
        #transforms.CenterCrop(224),
        transforms.ToTensor(),
        transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
    ]),
}

# data_dir = '/content/drive/MyDrive/Cifar10-VDSR(256)_split'
data_dir = '/content/drive/MyDrive/Cifar10_split'
image_datasets = {x: datasets.ImageFolder(os.path.join(data_dir, x),
                                          data_transforms[x])
                  for x in ['train', 'val', 'test']}
dataloaders = {x: torch.utils.data.DataLoader(image_datasets[x], batch_size=16,
                                             shuffle=True, num_workers=8, drop_last=True)
              for x in ['train', 'val', 'test']}
dataset_sizes = {x: len(image_datasets[x]) for x in ['train', 'val', 'test']}
class_names = image_datasets['train'].classes

device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

/usr/local/lib/python3.7/dist-packages/torch/utils/data/dataloader.py:481: UserWarning: This DataLoader will create 8 worker processes in total. Our suggested max number of worker in current system is 2, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  cpuset_checked))


In [22]:
class Net2(torch.nn.Module):
    def __init__(self, num_channels=3, upscale_factor=2, d=64, s=12, m=4):
        super(Net2, self).__init__()

        self.first_part = nn.Sequential(nn.Conv2d(in_channels=num_channels, out_channels=d, kernel_size=5, stride=1, padding=2),
                                        nn.PReLU())

        self.layers = []
        self.layers.append(nn.Sequential(nn.Conv2d(in_channels=d, out_channels=s, kernel_size=1, stride=1, padding=0),
                                         nn.PReLU()))
        for _ in range(m):
            self.layers.append(nn.Conv2d(in_channels=s, out_channels=s, kernel_size=3, stride=1, padding=1))
        self.layers.append(nn.PReLU())
        self.layers.append(nn.Sequential(nn.Conv2d(in_channels=s, out_channels=d, kernel_size=1, stride=1, padding=0),
                                         nn.PReLU()))

        self.mid_part = torch.nn.Sequential(*self.layers)

        # Deconvolution
        self.last_part = nn.ConvTranspose2d(in_channels=d, out_channels=num_channels, kernel_size=9, stride=upscale_factor, padding=3, output_padding=1)

    def forward(self, x):
        out = self.first_part(x)
        out = self.mid_part(out)
        out = self.last_part(out)
        return out

    def weight_init(self, mean=0.0, std=0.02):
        for m in self.modules():
            if isinstance(m, nn.Conv2d):
                m.weight.data.normal_(mean, std)
                if m.bias is not None:
                    m.bias.data.zero_()
            if isinstance(m, nn.ConvTranspose2d):
                m.weight.data.normal_(0.0, 0.0001)
                if m.bias is not None:
                    m.bias.data.zero_()


class Conv_ReLU_Block(nn.Module):
    def __init__(self):
        super(Conv_ReLU_Block, self).__init__()
        self.conv = nn.Conv2d(in_channels=128, out_channels=128, kernel_size=3, stride=1, padding=1, bias=False)
        self.relu = nn.ReLU(inplace=True)
        
    def forward(self, x):
        return self.relu(self.conv(x))
        
class Net(nn.Module):
    def __init__(self):
        super(Net, self).__init__()
        self.residual_layer = self.make_layer(Conv_ReLU_Block, 18)
        self.input = nn.Conv2d(in_channels=3, out_channels=128, kernel_size=3, stride=1, padding=1, bias=False)
        self.output = nn.Conv2d(in_channels=128, out_channels=3, kernel_size=3, stride=1, padding=1, bias=False)
        self.relu = nn.ReLU(inplace=True)
    
        for m in self.modules():
            if isinstance(m, nn.Conv2d):
                n = m.kernel_size[0] * m.kernel_size[1] * m.out_channels
                m.weight.data.normal_(0, sqrt(2. / n))
                
    def make_layer(self, block, num_of_layer):
        layers = []
        for _ in range(num_of_layer):
            layers.append(block())
        return nn.Sequential(*layers)

    def forward(self, x):
        residual = x
        out = self.relu(self.input(x))
        out = self.residual_layer(out)
        out = self.output(out)
        out = torch.add(out,residual)
        return residual,out

def conv_layer(chann_in, chann_out, k_size, p_size):
    layer = nn.Sequential(
        nn.Conv2d(chann_in, chann_out, kernel_size=k_size, padding=p_size),
        nn.BatchNorm2d(chann_out),
        nn.ReLU()
    )
    return layer

def vgg_fc_layer(size_in, size_out):
    layer = nn.Sequential(
        nn.Linear(size_in, size_out),
        nn.BatchNorm1d(size_out),
        nn.ReLU()
    )
    return layer

def vgg_conv_block(in_list, out_list, k_list, p_list, pooling_k, pooling_s):

    layers = [conv_layer(in_list[i], out_list[i], k_list[i], p_list[i]) for i in range(len(in_list))]
    layers += [nn.MaxPool2d(kernel_size=pooling_k, stride=pooling_s)]
    return nn.Sequential(*layers)

class SDCNN(nn.Module):
    def __init__(self, n_classes=10):
        super(SDCNN, self).__init__()

        # Conv blocks (BatchNorm + ReLU activation added in each block)
        self.layer1 = vgg_conv_block([3,32], [32,32], [3,3], [1,1], 2, 2)
        self.layer2 = vgg_conv_block([32,64], [64,64], [3,3], [1,1], 2, 2)

        # FC layers
        self.layer3 = vgg_fc_layer(16*16*64, 512)  # 4096->smaller

        # Final layer
        self.layer4 = nn.Linear(512, 10)

    def forward(self, x):
        # 从这里开始
        out = self.layer1(x)
        features = self.layer2(out)
        out = features.view(out.size(0), -1)
        out = self.layer3(out)
        out = self.layer4(out)

        return out


In [23]:
parser = argparse.ArgumentParser(description="PyTorch VDSR")
parser.add_argument("--nEpochs", type=int, default=50, help="Number of epochs to train for")
parser.add_argument("--lr", type=float, default=0.00001, help="Learning Rate. Default=0.1")
parser.add_argument("--step", type=int, default=10, help="Sets the learning rate to the initial LR decayed by momentum every n epochs, Default: n=10")
parser.add_argument("--cuda", action="store_true", help="Use cuda?")
parser.add_argument("--resume", default="",type=str, help="Path to checkpoint (default: none)")
parser.add_argument("--start-epoch", default=1, type=int, help="Manual epoch number (useful on restarts)")
parser.add_argument("--clip", type=float, default=0.4, help="Clipping Gradients. Default=0.4")
parser.add_argument("--threads", type=int, default=1, help="Number of threads for data loader to use, Default: 1")
parser.add_argument("--momentum", default=0.9, type=float, help="Momentum, Default: 0.9")
parser.add_argument("--weight-decay", "--wd", default=1e-4, type=float, help="Weight decay, Default: 1e-4")
parser.add_argument('--pretrained_SR', default='/content/drive/MyDrive/checkpoint/model_FSRCNN_CNET_*2_epoch_30.pth', type=str, help='path to pretrained model (default: none)')
parser.add_argument('--pretrained_TL', default='/content/drive/MyDrive/checkpoint/TransferLearning/model_FSRCNN_CNET_*2_epoch_30.pth', type=str, help='path to pretrained model (default: none)')
parser.add_argument("--gpus", default="0", type=str, help="gpu ids (default: 0)")
parser.add_argument("--pretrained_SR_num", default=1, type=int, help="numbers of epochs that have been trained")
parser.add_argument("--pretrained_TL_num", default=1, type=int, help="numbers of epochs that have been trained")
parser.add_argument("--SRtrain", default=True, help="if train the super resolution network")
parser.add_argument("--TLtrain", default=True, help="if train the trainsfer learning network")
parser.add_argument("--SR_used", default=True, help="if use the SR method thought the pipeline")

opt = parser.parse_args(args=[])

SRmodel = Net2()
CNET = SDCNN()
SRmodel = SRmodel.cuda()
CNET = CNET.cuda()

if opt.pretrained_SR:
    if os.path.isfile(opt.pretrained_SR):
        print("=> loading SR model '{}'".format(opt.pretrained_SR))
        weights = torch.load(opt.pretrained_SR)
        SRmodel.load_state_dict(weights['model'].state_dict())
    else:
        print("=> no model found at '{}'".format(opt.pretrained_SR))  
    
if opt.pretrained_TL:
    if os.path.isfile(opt.pretrained_TL):
        print("=> loading TL model '{}'".format(opt.pretrained_TL))
        weights = torch.load(opt.pretrained_TL)
        CNET.load_state_dict(weights['model'].state_dict())
    else:
        print("=> no model found at '{}'".format(opt.pretrained_TL))

correct = 0
total = 0
for images, labels in dataloaders['test']:
    images = images.cuda()
    if opt.SR_used:
      #_,images = SRmodel(images)
      images = SRmodel(images)
    outputs = CNET(images)
    _, predicted = torch.max(outputs.data, 1)
    total += labels.size(0)
    correct += (predicted.cpu() == labels).sum()
    print("avg acc: %f" % (100* correct/total))

=> loading SR model '/content/drive/MyDrive/checkpoint/model_FSRCNN_CNET_*2_epoch_30.pth'
=> loading TL model '/content/drive/MyDrive/checkpoint/TransferLearning/model_FSRCNN_CNET_*2_epoch_30.pth'


/usr/local/lib/python3.7/dist-packages/torch/utils/data/dataloader.py:481: UserWarning: This DataLoader will create 8 worker processes in total. Our suggested max number of worker in current system is 2, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  cpuset_checked))


avg acc: 87.500000
avg acc: 78.125000
avg acc: 77.083336
avg acc: 76.562500
avg acc: 80.000000
avg acc: 82.291664
avg acc: 82.142860
avg acc: 78.125000
avg acc: 79.166664
avg acc: 78.750000
avg acc: 80.113640
avg acc: 80.729164
avg acc: 78.846153
avg acc: 78.571426
avg acc: 78.750000
avg acc: 78.515625
avg acc: 77.941177
avg acc: 77.777779
avg acc: 77.631577
avg acc: 78.125000
avg acc: 77.976189
avg acc: 78.125000
avg acc: 77.989128
avg acc: 78.125000
avg acc: 78.000000
avg acc: 77.884613
avg acc: 77.083336
avg acc: 77.678574
avg acc: 77.801727
avg acc: 77.708336
avg acc: 78.024193
avg acc: 77.929688
avg acc: 77.840912
avg acc: 77.757355
avg acc: 77.678574
avg acc: 78.298615
avg acc: 77.871620
avg acc: 77.960526
avg acc: 78.044868
avg acc: 77.812500
avg acc: 77.896339
avg acc: 77.827377
avg acc: 78.197678
avg acc: 78.267044
avg acc: 78.750000
avg acc: 78.804344
avg acc: 78.723404
avg acc: 78.906250
avg acc: 78.954079
avg acc: 79.000000
avg acc: 79.044121
avg acc: 79.206734
avg acc: 79.